In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Composition, abstraction, and class invariants

Compared to aggregation composition is more costly in terms of memory and time because of the need to make defensive copies of fields and parameters. The extra expense of memory and time imposed by composition is often required to support the abstraction and/or invariants provided by a class.

## Composition and abstraction

We've already seen an example where composition is required to support an abstraction implemented by a class. Recall the `Stack` class from the [Anatomy of a simple class](../part2/anatomy_of_a_simple_class.ipynb#notebook_id) notebook:

In [ ]:
import java.util.ArrayList;

public class Stack {

    private ArrayList<String> stack;
    
    public Stack() {
        this.stack = new ArrayList<>();
    }

    public int size() {
        return this.stack.size();
    }

    public void push(String s) {
        this.stack.add(s);
    }

    public String pop() {
        String s = this.stack.remove(this.size() - 1);
        return s;
    }

    public String toString() {
        StringBuilder b = new StringBuilder("ListStack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >=0; i--) {
                b.append('\n');
                b.append(this.stack.get(i));
            }
        }
        return b.toString();
    }
}

The reader should pause to review the implementation of the class and confirm that a `Stack` is a composition that has an `ArrayList` as its part.

**Exercise 1** Verify that the three rules for implementing composition described in the [Association, aggregation, and composition](./association.ipynb#notebook_id) notebook are followed in the `Stack` class.

**Exercise 2** Argue that a `Stack` is an aggregation of 0 or more strings. Why is it not a composition of 0 or more strings?

Recall that the stack data structure is a collection of elements where the elements can be inserted and removed, but removing an element always causes the most recently inserted element to be removed. The insert and remove operations are called push and pop, respectively, and the stack abstraction allows elements to be added and removed from the stack only using these operations. Pushing an element adds the element to the top of the stack and popping an element removes the element at the top of the stack.

Consider what happens when we break the composition by exposing a reference to its `ArrayList` part:

In [ ]:
import java.util.ArrayList;

public class BrokenStack {

    private ArrayList<String> stack;
    
    public BrokenStack() {
        this.stack = new ArrayList<>();
    }

    public int size() {
        return this.stack.size();
    }

    public void push(String s) {
        this.stack.add(s);
    }

    public String pop() {
        String s = this.stack.remove(this.size() - 1);
        return s;
    }
    
    // oops, expose a reference to the ArrayList part
    public ArrayList<String> getElements() {
        return this.stack;
    }

    public String toString() {
        StringBuilder b = new StringBuilder("BrokenStack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >=0; i--) {
                b.append('\n');
                b.append(this.stack.get(i));
            }
        }
        return b.toString();
    }
}

The `BrokenStack` class no longer provides the abstraction of a stack because the user can add or remove elements to the stack anywhere they want instead of only at the top of the stack:

In [ ]:
BrokenStack s = new BrokenStack();
s.push("bottom");
s.push("x");
s.push("y");
s.push("top");
System.out.println(s);
System.out.println();

// add an element to the middle of the stack
s.getElements().add(2, "middle");
System.out.println(s);
System.out.println();

// remove the bottom element of the stack
s.getElements().remove(0);
System.out.println(s);

**Exercise 3** Add a constructor to the `BrokenStack` class that has an `ArrayList<String>` as one of its parameters and show how an incorrectly implemented constructor can break a composition.

## Composition and invariants

Recall that a class invariant is a condition regarding the state of an object that is guaranteed to be true immediately after an object is initialized and immediately after a public method is invoked using the object. If an invariant depends on the state of a mutable reference type field then composition is probably required to ensure that the invariant is preserved.

Consider the following example which is adapted from *Effective Java* by Joshua Bloch (available online at http://www.informit.com/articles/article.aspx?p=31551&seqNum=2).

We want to implement a class that represents a period of time. Such a class can be used to represent an appointment time where the period has a start time and an end time. An invariant of a time period is that the end time is always after the start time. To represent a time we will use the class `java.util.Date` that represents a specific instant in time, with millisecond precision. The `Date` class is an older class and has some unusual quirks; at the end of this notebook is an alternative implementation that uses the `java.time.LocalTime` class. One of the quirks of the `Date` class is that it has no methods or constructors that lets the programmer easily make a copy of an existing `Date` object. Copying a `Date` instance can be done as follows:

In [ ]:
import java.util.Date;

Date d = new Date();  // the date and time now
Date copy = new Date(d.getTime());
System.out.println("d: " + d);
System.out.println("copy: " + copy);

### `Period` as an aggregate

We start by showing an implementation that uses aggregation instead of composition, illustrate why aggregation cannot be used if the class invariants are to be preserved, and then correct the implementation to use composition.

The UML diagram for the `Period` class that is implemented as an aggregate is shown below:

![Period class as aggregate](../resources/images/part3/period_agg.png)

Note that the `Period` class has a method `isInvariantTrue` that returns `true` if the class invariant is true that we will use to test if the class invariant is true.

In [ ]:
import java.util.Date;

public class Period {
    private Date start;
    private Date end;

    /**
     * Initialize the period to the given start and end dates.
     * 
     * @param start beginning of the period
     * @param end end of the period; must not precede start
     * @throws IllegalArgumentException if start is after end
     */
    public Period(Date start, Date end) {
        if (start.compareTo(end) > 0) {
            throw new IllegalArgumentException("start after end");
        }
        this.start = start;
        this.end = end;
    } 

    /**
     * Initialize the period so that it has the same start and end times
     * as the specified period.
     *
     * @param other the period to copy
     */
    public Period(Period other) {
        this.start = other.start;
        this.end = other.end;
    } 

    /**
     * Returns the starting date of the period.
     * 
     * @return the starting date of the period
     */
    public Date getStart() {
        return this.start;
    }

    /**
     * Returns the ending date of the period.
     * 
     * @return the ending date of the period
     */
    public Date getEnd() {
        return this.end;
    }

    /**
     * Sets the starting date of the period.
     * 
     * @param newStart the new starting date of the period
     * @return true if the new starting date is earlier than the
     *         current end date; false otherwise
     */
    public boolean setStart(Date newStart) {
        boolean ok = false;
        if (newStart.compareTo(this.end) < 0) {
            this.start = newStart;
            ok = true;
        }
        return ok;
    }

    /**
     * Sets the ending date of the period.
     * 
     * @param newEnd the new ending date of the period
     * @return true if the new ending date is after the
     *         current start date; false otherwise
     */
    public boolean setEnd(Date newEnd) {
        boolean ok = false;
        if (newEnd.compareTo(this.start) > 0) {
            this.end = newEnd;
            ok = true;
        }
        return ok;
    }
    
    /**
     * Extend the duration of this period by the specified number of
     * hours. The period is extended by changing the end time of
     * this period.
     *
     * @param hours the number of hours to extend this period by
     */
    public void extend(int hours) {
        if (hours < 0) {
            throw new IllegalArgumentException();
        }
        this.end.setHours(this.end.getHours() + hours);
    }
    
    /**
     * Returns true if the start time of this period is before 
     * the end time of this period.
     *
     * @return true if the class invariant is true
     */
    public boolean isInvariantTrue() {
        return this.start.compareTo(this.end) < 0;
    }

    /**
     * Compares this period with an object for equality. The result
     * is true if and only if obj is a Period object having an equal
     * start and end time as this period.
     *
     * @param obj the object to compare
     * @return true if obj is equal to this period
     */
    public boolean equals(Object obj) {
        if (this == obj) {
            return true;
        }
        if (!(obj instanceof Period)) {
            return false;
        }
        Period other = (Period) obj;
        return this.start.equals(other.start) && this.end.equals(other.end);
    }
}

**Exercise 4** Identify all of the privacy leaks in the `Period` implementation shown above.

The first constructor fails to create defensive copies of the parameters `start` and `end` which creates a privacy leak. A user can can break the class invariant of `Period` by mutating the state of the user-supplied starting time:

In [ ]:
import java.util.Date;

// 14:30 on Nov 1, 2020
int yr = 2020 - 1900;
int mn = 11;
int day = 1;
int hr = 14;
int min = 30;
Date start = new Date(yr, mn, day, hr, min);

// 15:30 on Nov 1, 2020
Date end = new Date(start.getTime());
end.setHours(hr + 1);

Period p = new Period(start, end);
System.out.println("invariant true?: " + p.isInvariantTrue());

// change state of start
start.setHours(hr + 2);
System.out.println("invariant true?: " + p.isInvariantTrue());

Notice that the class invariant can be broken by a user without even using a method in the `Period` class; all that is required to break the invariant is to mutate the state of `start`.

**Exercise 5** Show how to break the class invariant by manipulating the `end` time.

The copy constructor fails to create defensive copies of the copied `Period`'s start and end times (`other.start` and `other.end`, respectively) which creates a privacy leak. This causes the original and copied object to behave as though they were the same object:

In [ ]:
import java.util.Date;

// 14:30 on Nov 1, 2020
int yr = 2020 - 1900;
int mn = 11;
int day = 1;
int hr = 14;
int min = 30;
Date start = new Date(yr, mn, day, hr, min);

// 15:30 on Nov 1, 2020
Date end = new Date(start.getTime());
end.setHours(hr + 1);

Period p = new Period(start, end);

// copy p
Period q = new Period(p);
System.out.println("q equals p?: " + q.equals(p));

// change state of q by extending the duration of the period by 1 hour
q.extend(1);
System.out.println("q equals p?: " + q.equals(p));


Notice that changing the duration of period `q` also changes the duration of period `p`; this occurs because `p` and `q` share the same start and end time `Date` objects.

The accessor method `getStart` directly returns the field `this.start` which is a privacy leak. This allows a user to change the start time of a `Period` object without invoking a method using the object:

In [ ]:
import java.util.Date;

// 14:30 on Nov 1, 2020
int yr = 2020 - 1900;
int mn = 11;
int day = 1;
int hr = 14;
int min = 30;
Date start = new Date(yr, mn, day, hr, min);

// 15:30 on Nov 1, 2020
Date end = new Date(start.getTime());
end.setHours(hr + 1);

Period p = new Period(start, end);
System.out.println("invariant true?: " + p.isInvariantTrue());

// get the start time
Date pStart = p.getStart();

// change pStart
pStart.setHours(hr + 2);
System.out.println("invariant true?: " + p.isInvariantTrue());

The `Period` object `p` cannot guarantee that its invariant is true because `pStart` refers to the same `Date` object that represents `p`'s starting time and the user is free to mutate the state of the `Date` object using `pStart`.

**Exercise 6** Show how to break the class invariant by getting the end time of a period.

The mutator method `setStart` fails to assign a defensive copy of its parameter `newStart` to its field `this.start` which is a privacy leak. A user can can break the class invariant of `Period` by mutating the state of the user-supplied new starting time:

In [ ]:
import java.util.Date;

// 14:30 on Nov 1, 2020
int yr = 2020 - 1900;
int mn = 11;
int day = 1;
int hr = 14;
int min = 30;
Date start = new Date(yr, mn, day, hr, min);

// 15:30 on Nov 1, 2020
Date end = new Date(start.getTime());
end.setHours(hr + 1);

Period p = new Period(start, end);
System.out.println("invariant true?: " + p.isInvariantTrue());

// new start time (earlier than original start time)
Date newStart = new Date(start.getTime());
newStart.setHours(hr - 1);

// change start time using setStart
p.setStart(newStart);
System.out.println("invariant true?: " + p.isInvariantTrue());

// mutate newStart
newStart.setHours(hr + 3);
System.out.println("invariant true?: " + p.isInvariantTrue());

Notice that calling `setStart` does not break the class invariant directly (as indicated by `true` being output twice). The invariant is broken when the state of `setStart` is changed.

**Exercise 7** Show how to break the class invariant by setting a new end time of a period.


### `Period` as a composite

All of the problems illustrated in the previous examples are caused because the `Period` class fails to ensure that its fields are not aliases. The problems are prevented by making defensive copies at the appropriate times. 

The UML diagram for the `Period` class that is implemented as a composite is shown below:

![Period class as composite](../resources/images/part3/period_comp.png)

The next cell shows a `Period` class that correctly implements a composition.

**Exercise 8** Run the next cell to compile the corrected version of the `Period` class. Then re-run the previous examples and verify that the class invariant is always preserved. In the copy constructor example, verify that `p` and `q` are not equal after the duration of `q` is extended.

In [ ]:
import java.util.Date;

public class Period {
    private Date start;
    private Date end;

    /**
     * Initialize the period to the given start and end dates.
     * 
     * @param start beginning of the period
     * @param end end of the period; must not precede start
     * @throws IllegalArgumentException if start is after end
     */
    public Period(Date start, Date end) {
        if (start.compareTo(end) > 0) {
            throw new IllegalArgumentException("start after end");
        }
        this.start = new Date(start.getTime());
        this.end = new Date(end.getTime());
    } 

    /**
     * Initialize the period so that it has the same start and end times
     * as the specified period.
     *
     * @param other the period to copy
     */
    public Period(Period other) {
        this.start = new Date(other.start.getTime());
        this.end = new Date(other.end.getTime());
    } 

    /**
     * Returns the starting date of the period.
     * 
     * @return the starting date of the period
     */
    public Date getStart() {
        return new Date(this.start.getTime());
    }

    /**
     * Returns the ending date of the period.
     * 
     * @return the ending date of the period
     */
    public Date getEnd() {
        return new Date(this.end.getTime());
    }

    /**
     * Sets the starting date of the period.
     * 
     * @param newStart the new starting date of the period
     * @return true if the new starting date is earlier than the
     *         current end date; false otherwise
     */
    public boolean setStart(Date newStart) {
        boolean ok = false;
        Date copy = new Date(newStart.getTime());
        if (copy.compareTo(this.end) < 0) {
            this.start = copy;
            ok = true;
        }
        return ok;
    }

    /**
     * Sets the ending date of the period.
     * 
     * @param newEnd the new ending date of the period
     * @return true if the new ending date is after the
     *         current start date; false otherwise
     */
    public boolean setEnd(Date newEnd) {
        boolean ok = false;
        Date copy = new Date(newEnd.getTime());
        if (copy.compareTo(this.start) > 0) {
            this.end = copy;
            ok = true;
        }
        return ok;
    }
    
    /**
     * Extend the duration of this period by the specified number of
     * hours. The period is extended by changing the end time of
     * this period.
     *
     * @param hours the number of hours to extend this period by
     */
    public void extend(int hours) {
        if (hours < 0) {
            throw new IllegalArgumentException();
        }
        this.end.setHours(this.end.getHours() + hours);
    }
    
    /**
     * Returns true if the start time of this period is before 
     * the end time of this period.
     *
     * @return true if the class invariant is true
     */
    public boolean isInvariantTrue() {
        return this.start.compareTo(this.end) < 0;
    }

    /**
     * Compares this period with an object for equality. The result
     * is true if and only if obj is a Period object having an equal
     * start and end time as this period.
     *
     * @param obj the object to compare
     * @return true if obj is equal to this period
     */
    public boolean equals(Object obj) {
        if (this == obj) {
            return true;
        }
        if (!(obj instanceof Period)) {
            return false;
        }
        Period other = (Period) obj;
        return this.start.equals(other.start) && this.end.equals(other.end);
    }
}

## An alternate implementation of the `Period` class

The reader may have noticed that the reason composition is required to preserve the class invariant in the `Period` class is because the type (`Date`) used to represent the start and end time of the period is mutable. All of the examples where the user breaks the invariant requires the user to mutate the state of a `Date` object.

If an immutable type is used to represent the start and end times of a period then composition is no longer required to preserve the invariants of the `Period` class. The immutable types `java.time.LocalTime` or `java.time.LocalDateTime` can be used in place of the `Date` class.

The following cell contains an implementation of the `Period` class where a `Period` is an aggregation of two `LocalTime` objects. The cell after the following cell is a modified version of one of the earlier examples that shows that the modified class preserves the class invariant.

Because `LocalTime` is immutable, there is no need to create defensive copies which simplifies the implementation of the class. 
The lesson here is that immutable objects are good building blocks (or parts) for other objects because immutable objects can be shared freely without the risk that their state might change.

**Exercise 9** Under what situation is an immutable object not an appropriate building block for another object?

In [ ]:
import java.time.LocalTime;

public class Period {
    private LocalTime start;
    private LocalTime end;

    /**
     * Initialize the period to the given start and end times.
     * 
     * @param start beginning of the period
     * @param end end of the period; must not precede start
     * @throws IllegalArgumentException if start is after end
     */
    public Period(LocalTime start, LocalTime end) {
        if (start.compareTo(end) > 0) {
            throw new IllegalArgumentException("start after end");
        }
        this.start = start;
        this.end = end;
    } 

    /**
     * Initialize the period so that it has the same start and end times
     * as the specified period.
     *
     * @param other the period to copy
     */
    public Period(Period other) {
        this.start = other.start;
        this.end = other.end;
    } 

    /**
     * Returns the starting time of the period.
     * 
     * @return the starting time of the period
     */
    public LocalTime getStart() {
        return this.start;
    }

    /**
     * Returns the ending time of the period.
     * 
     * @return the ending time of the period
     */
    public LocalTime getEnd() {
        return this.end;
    }

    /**
     * Sets the starting time of the period.
     * 
     * @param newStart the new starting time of the period
     * @return true if the new starting time is earlier than the
     *         current end time; false otherwise
     */
    public boolean setStart(LocalTime newStart) {
        boolean ok = false;
        if (newStart.isBefore(this.end)) {
            this.start = newStart;
            ok = true;
        }
        return ok;
    }

    /**
     * Sets the ending time of the period.
     * 
     * @param newEnd the new ending time of the period
     * @return true if the new ending time is after the
     *         current start time; false otherwise
     */
    public boolean setEnd(LocalTime newEnd) {
        boolean ok = false;
        if (newEnd.isAfter(this.start)) {
            this.end = newEnd;
            ok = true;
        }
        return ok;
    }
    
    /**
     * Extend the duration of this period by the specified number of
     * hours. The period is extended by changing the end time of
     * this period.
     *
     * @param hours the number of hours to extend this period by
     */
    public void extend(int hours) {
        if (hours < 0) {
            throw new IllegalArgumentException();
        }
        this.end = this.end.plusHours(hours);
    }
    
    /**
     * Returns true if the start time of this period is before 
     * the end time of this period.
     *
     * @return true if the class invariant is true
     */
    public boolean isInvariantTrue() {
        return this.start.isBefore(this.end);
    }

    /**
     * Compares this period with an object for equality. The result
     * is true if and only if obj is a Period object having an equal
     * start and end time as this period.
     *
     * @param obj the object to compare
     * @return true if obj is equal to this period
     */
    public boolean equals(Object obj) {
        if (this == obj) {
            return true;
        }
        if (!(obj instanceof Period)) {
            return false;
        }
        Period other = (Period) obj;
        return this.start.equals(other.start) && this.end.equals(other.end);
    }
}

In [ ]:
import java.time.LocalTime;

// 14:30
int hr = 14;
int min = 30;
LocalTime start = LocalTime.of(hr, min);

// 15:30
LocalTime end = LocalTime.of(hr + 1, min);

Period p = new Period(start, end);
System.out.println("invariant true?: " + p.isInvariantTrue());

// new start time (earlier than original start time)
LocalTime newStart = LocalTime.of(hr - 1, min);

// change start time using setStart
p.setStart(newStart);
System.out.println("invariant true?: " + p.isInvariantTrue());

// assign a new time to newStart
newStart = LocalTime.of(hr + 3, min);
System.out.println("invariant true?: " + p.isInvariantTrue());